# 2 start base pods on minikube

change ${PJ_ROOT} to your directory.

In [ ]:
export PJ_ROOT="${HOME}/core"
cd ${PJ_ROOT};pwd

example)
```
/Users/user/core
```

## load environment variables

In [ ]:
source ${PJ_ROOT}/docs/environments/minikube/env

## setup alias

In [ ]:
if [ "$(uname)" == 'Darwin' ]; then
  alias randomstr32='cat /dev/urandom | LC_CTYPE=C tr -dc 'a-zA-Z0-9' | head -c 32'
elif [ "$(expr substr $(uname -s) 1 5)" == 'Linux' ]; then
  alias randomstr32='cat /dev/urandom 2>/dev/null | head -n 40 | tr -cd 'a-zA-Z0-9' | head -c 32'
else
  echo "Your platform ($(uname -a)) is not supported."
  exit 1
fi

## start RabbitMQ cluster on minikube
[rabbitmq](https://www.rabbitmq.com/)

### start RabbitMQ

In [ ]:
kubectl apply -f rabbitmq/rabbitmq-rbac.yaml

In [ ]:
kubectl apply -f rabbitmq/rabbitmq-minikube-services.yaml

In [ ]:
kubectl apply -f rabbitmq/rabbitmq-minikube-statefulset.yaml

In [ ]:
kubectl get pods -l app=rabbitmq

example)

```
NAME         READY     STATUS    RESTARTS   AGE
rabbitmq-0   1/1       Running   0          6m
rabbitmq-1   1/1       Running   0          5m
rabbitmq-2   1/1       Running   0          4m
```

In [ ]:
kubectl exec rabbitmq-0 -- rabbitmqctl cluster_status

example)
```
Cluster status of node rabbit@rabbitmq-0.rabbitmq.default.svc.cluster.local ...
[{nodes,[{disc,['rabbit@rabbitmq-0.rabbitmq.default.svc.cluster.local',
                'rabbit@rabbitmq-1.rabbitmq.default.svc.cluster.local',
                'rabbit@rabbitmq-2.rabbitmq.default.svc.cluster.local']}]},
 {running_nodes,['rabbit@rabbitmq-2.rabbitmq.default.svc.cluster.local',
                 'rabbit@rabbitmq-1.rabbitmq.default.svc.cluster.local',
                 'rabbit@rabbitmq-0.rabbitmq.default.svc.cluster.local']},
 {cluster_name,<<"rabbit@rabbitmq-0.rabbitmq.default.svc.cluster.local">>},
 {partitions,[]},
 {alarms,[{'rabbit@rabbitmq-2.rabbitmq.default.svc.cluster.local',[]},
          {'rabbit@rabbitmq-1.rabbitmq.default.svc.cluster.local',[]},
          {'rabbit@rabbitmq-0.rabbitmq.default.svc.cluster.local',[]}]}]
```

### change password of `guest`

In [ ]:
kubectl exec rabbitmq-0 -- rabbitmqctl change_password guest $(randomstr32)

### register users to RabbitMQ

In [ ]:
for e in $(env); do
  if [[ "${e}" =~ ^MQTT__([[:alnum:]_-]+)=([[:alnum:]_-]+)$ ]]; then
    username=${BASH_REMATCH[1]}
    password=${BASH_REMATCH[2]}
    
    kubectl exec rabbitmq-0 -- rabbitmqctl add_user ${username} ${password}
    kubectl exec rabbitmq-0 -- rabbitmqctl set_permissions -p / ${username} ".*" ".*" ".*"
  fi
done

In [ ]:
kubectl exec rabbitmq-0 -- rabbitmqctl list_users

example)
```
Listing users ...
guest	[administrator]
iotagent	[]
```

In [ ]:
kubectl get services -l app=rabbitmq

expected)
```
NAME            TYPE        CLUSTER-IP       EXTERNAL-IP   PORT(S)              AGE
rabbitmq        ClusterIP   None             <none>        5672/TCP             4m
rabbitmq-amqp   ClusterIP   10.107.253.173   <none>        15672/TCP,5672/TCP   4m
rabbitmq-mqtt   NodePort    10.102.140.44    <none>        1883:31186/TCP       4m
```

### nat MQTT port 

In [ ]:
export MQTT_PORT=$(kubectl describe service rabbitmq-mqtt | grep "NodePort:" | awk '{print $3}' | awk -F'/' '{print $1}');echo ${MQTT_PORT}

In [ ]:
VBoxManage controlvm "${MINIKUBE_NAME}" natpf1 "mqtt,tcp,0.0.0.0,1883,,${MQTT_PORT}"
VBoxManage showvminfo "${MINIKUBE_NAME}"| grep ${MQTT_PORT}

example)
```
NIC 1 Rule(0):   name = mqtt, protocol = tcp, host ip = 0.0.0.0, host port = 1883, guest ip = , guest port = 30382
```

In [ ]:
export WEB_MQTT_PORT=$(kubectl describe service rabbitmq-web-mqtt | grep "NodePort:" | awk '{print $3}' | awk -F'/' '{print $1}');echo ${WEB_MQTT_PORT}

In [ ]:
VBoxManage controlvm "${MINIKUBE_NAME}" natpf1 "web-mqtt,tcp,0.0.0.0,15675,,${WEB_MQTT_PORT}"
VBoxManage showvminfo "${MINIKUBE_NAME}"| grep ${WEB_MQTT_PORT}

### confirm to connect MQTT broker

In [ ]:
mosquitto_pub -h ${HOST_IPADDR} -p 1883  -d -u iotagent -P ${MQTT__iotagent} -t /test -m "test"

example)
```
Client mosqpub|52150-Nobuyukin sending CONNECT
Client mosqpub|52150-Nobuyukin received CONNACK
Client mosqpub|52150-Nobuyukin sending PUBLISH (d0, q0, r0, m1, '/test', ... (4 bytes))
Client mosqpub|52150-Nobuyukin sending DISCONNECT
```

## start mondodb cluster on minikube
[mongodb](https://www.mongodb.com/)

create 'mongodb' takes a few minutes.

In [ ]:
helm install --name mongodb -f mongodb/mongodb-replicaset-values-minikube.yaml stable/mongodb-replicaset

In [ ]:
kubectl get statefulsets -l release=mongodb -l app=mongodb-replicaset

example)
```
NAME      READY   AGE
mongodb   3/3     106s
```

In [ ]:
kubectl get pods -l release=mongodb -l app=mongodb-replicaset

example)
```
NAME        READY   STATUS    RESTARTS   AGE
mongodb-0   1/1     Running   0          105s
mongodb-1   1/1     Running   0          38s
mongodb-2   1/1     Running   0          24s
```

In [ ]:
kubectl get services -l release=mongodb -l app=mongodb-replicaset

example)
```
NAME             TYPE        CLUSTER-IP   EXTERNAL-IP   PORT(S)     AGE
mongodb          ClusterIP   None         <none>        27017/TCP   2m21s
mongodb-client   ClusterIP   None         <none>        27017/TCP   2m21s
```

In [ ]:
kubectl exec mongodb-0 -c mongodb-replicaset -- mongo --eval 'printjson(rs.status().members.map(function(e) {return {name: e.name, stateStr:e.stateStr};}))'

expected)
```
MongoDB shell version v4.1.13
connecting to: mongodb://127.0.0.1:27017/?compressors=disabled&gssapiServiceName=mongodb
Implicit session: session { "id" : UUID("21eaa109-63dd-4b65-88b4-e6ba9f8bb338") }
MongoDB server version: 4.1.13
[
	{
		"name" : "mongodb-0.mongodb.default.svc.cluster.local:27017",
		"stateStr" : "PRIMARY"
	},
	{
		"name" : "mongodb-1.mongodb.default.svc.cluster.local:27017",
		"stateStr" : "SECONDARY"
	},
	{
		"name" : "mongodb-2.mongodb.default.svc.cluster.local:27017",
		"stateStr" : "SECONDARY"
	}
]
```

## start ambassador on minikube

[ambassador](https://www.getambassador.io/)

### start ambassador

In [ ]:
kubectl apply -f ambassador/ambassador-minikube-service.yaml

In [ ]:
kubectl apply -f ambassador/ambassador-deployment.yaml

In [ ]:
kubectl get pods -l app=ambassador

example)
```
NAME                         READY     STATUS    RESTARTS   AGE
ambassador-cf99ff6bb-45jpl   1/1       Running   0          39s
ambassador-cf99ff6bb-8kht7   1/1       Running   0          39s
ambassador-cf99ff6bb-lv2tk   1/1       Running   0          39s
```

In [ ]:
kubectl get services -l app=ambassador

example)
```
NAME         TYPE       CLUSTER-IP      EXTERNAL-IP   PORT(S)        AGE
ambassador   NodePort   10.98.181.146   <none>        80:31807/TCP   1m
```

### nat HTTP port

In [ ]:
HTTP_PORT=$(kubectl describe service ambassador | grep "NodePort:" | awk '{print $3}' | awk -F'/' '{print $1}');echo ${HTTP_PORT}

In [ ]:
VBoxManage controlvm "${MINIKUBE_NAME}" natpf1 "http,tcp,0.0.0.0,8080,,${HTTP_PORT}"
VBoxManage showvminfo "${MINIKUBE_NAME}" | grep ${HTTP_PORT}

In [ ]:
curl -i http://${HOST_IPADDR}:8080

expected)
```
HTTP/1.1 404 Not Found
date: Fri, 25 May 2018 00:47:41 GMT
server: envoy
content-length: 0
```